In [218]:
import pandas as pd
#from urllib.request import urlretrieve
#import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import torchvision
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
import time
import torch
import glob
from sklearn.preprocessing import StandardScaler

In [219]:
torch.cuda.is_available()
def get_device():
    # Step 2: Check if a GPU is available
    if torch.cuda.is_available():
        # Step 3: Set the device to GPU
        device = torch.device("cuda")
        print("GPU is available. Using GPU.")
    else:
        # Step 3: Default to CPU
        device = torch.device("cpu")
        print("GPU is not available. Using CPU.")
    
    # Step 4: Return the device
    return device

# Example usage
device = get_device()

GPU is available. Using GPU.


In [220]:
class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerNN, self).__init__()
        # Define the layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # First fully connected layer
        self.relu1 = nn.ReLU()  # Activation function
        self.fc2 = nn.Linear(hidden_size, hidden_size)  # Second fully connected layer
        self.relu2 = nn.ReLU()  # Activation function
        self.fc3 = nn.Linear(hidden_size, hidden_size)  # Third fully connected layer
        self.relu3 = nn.ReLU()  # Activation function
        self.fc4 = nn.Linear(hidden_size, output_size)  # fourth fully connected layer

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

In [221]:

#num_samples = 1000
#num_instances = 10000


#x = torch.rand(num_instances, num_samples)
#y = torch.rand(num_instances, num_samples)
#z = torch.rand(num_instances, num_samples)
#target = torch.randint(-1000, 1001, (num_instances, num_samples))
#data = torch.stack((x, y, z, target), dim=2).to(device)
# Define the pattern to match files
start_time = time.time()
file_pattern = 'SWPR_WDFC.wRes.nom_SP.058_SP.060_2D_w0(*).xlsx'

# Find all files matching the pattern
file_paths = glob.glob(file_pattern)

# Initialize a list to store the tensors
data_list = []

# Loop through the list of matching files
for file_path in file_paths:
    columns_to_read = ['E_X', 'E_Y', 'E_Z', 'PSCL']
    df = pd.read_excel(file_path, usecols=columns_to_read)
    data_array = df.to_numpy()
    data_tensor = torch.tensor(data_array, dtype=torch.float32)
    data_list.append(data_tensor)

# Determine the maximum size of the second dimension
max_size = max(tensor.shape[0] for tensor in data_list)

# Pad each tensor to the maximum size
padded_data_list = []
for tensor in data_list:
    padding = (0, 0, 0, max_size - tensor.shape[0])
    padded_tensor = torch.nn.functional.pad(tensor, padding, mode='constant', value=0)
    padded_data_list.append(padded_tensor)

# Stack the padded tensors into a single tensor
data_unscaled = torch.stack(padded_data_list)

# Print the shape of the final tensor

print(data_unscaled.shape)

#data scaling+++++++++++++++++++++++++++++++++++++++++++++++++++++++++
data_2d = data_unscaled.view(-1, 4).numpy()
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the 2D data
scaler.fit(data_2d)
# Transform the data
scaled_data_2d = scaler.transform(data_2d)
# Reshape the scaled data back to 3D
data = torch.tensor(scaled_data_2d).view(len(file_paths), max_size, 4)



end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for reading data")
print(data.shape)





torch.Size([10, 5670, 4])
Elapsed time: 36.30769348144531 seconds for reading data
torch.Size([10, 5670, 4])


In [222]:
#print("origincal data:")
#print(data_unscaled)
print("Scaled data:")
print(data)

Scaled data:
tensor([[[ 1.3760e+00,  7.9743e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3746e+00,  8.0359e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3733e+00,  8.0975e-01,  0.0000e+00,  4.9289e-01],
         ...,
         [ 3.3940e-01, -3.6861e-01,  0.0000e+00,  4.7618e-02],
         [ 3.3940e-01, -3.7060e-01,  0.0000e+00,  5.2671e-02],
         [ 3.3940e-01, -3.7258e-01,  0.0000e+00,  5.7675e-02]],

        [[ 1.3760e+00,  7.9743e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3746e+00,  8.0359e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3733e+00,  8.0975e-01,  0.0000e+00,  4.9289e-01],
         ...,
         [ 3.3940e-01, -3.6861e-01,  0.0000e+00,  4.7618e-02],
         [ 3.3940e-01, -3.7060e-01,  0.0000e+00,  5.2671e-02],
         [ 3.3940e-01, -3.7258e-01,  0.0000e+00,  5.7675e-02]],

        [[ 1.3760e+00,  7.9743e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3746e+00,  8.0359e-01,  0.0000e+00,  4.9289e-01],
         [ 1.3733e+00,  8.0975e-01,  0.0000e+00,  4.9289e-01],
         .

In [223]:
val_size = len(data_list)//10
train_size = len(data_list) - val_size
print(val_size)
print(train_size)
start_time = time.time()
train_ds, val_ds = random_split(data, [train_size, val_size])
#batch_size=2
#train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0)
#val_loader = DataLoader(val_ds, batch_size*2, num_workers=0)

train_loader  = torch.stack([data[i] for i in train_ds.indices])
val_loader = torch.stack([data[i] for i in val_ds.indices])
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for manipulating data.")

1
9
Elapsed time: 0.0 seconds for manipulating data.


In [224]:
#for batch in train_loader:
#    print(batch.shape)
#    inputs =    batch[:,...,:-1]
#    targets =   batch[:,...,-1]
#    print(inputs)
#    print(targets)   # Forward pass
#    break
train_loader.shape

torch.Size([9, 5670, 4])

In [225]:
model = ThreeLayerNN(3, 100, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [226]:


# Assuming model, train_loader, and criterion are already defined
#model = ...  # Your neural network model
#train_loader = ...  # DataLoader for your training data
criterion = nn.MSELoss()  # Loss function for regression

num_epochs = 10000  # Number of epochs

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    optimizer.zero_grad()  # Zero the gradients
    inputs = train_loader[:,...,:-1].to(device)
    outputs = model(inputs).to(device)
    targets =   train_loader[:,...,-1].to(device)   # Forward pass
    outputs = outputs.squeeze(-1)
    loss = criterion(outputs, targets)  # Compute the loss

    loss.backward()  # Backward pass
    optimizer.step()  # Update the weights

    running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}")
    #VALI
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0

    with torch.no_grad():
    
        inputs = val_loader[:, ..., :-1].to(device)
        outputs = model(inputs).to(device)
        targets = val_loader[:, ..., -1].to(device)  # Forward pass
        outputs = outputs.squeeze(-1)
        loss = criterion(outputs, targets).to(device)  # Compute the loss
        val_loss += loss.item()
        print(f" Validation Loss: {val_loss/len(val_loader)}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds for {len(data_list)} simulations with {len(data[1])} points each on {device}.")


Epoch [1/10000], Loss: 0.11140926678975423
 Validation Loss: 1.0010123252868652
Epoch [2/10000], Loss: 0.11138368977440728
 Validation Loss: 1.0007871389389038
Epoch [3/10000], Loss: 0.11135823196834987
 Validation Loss: 1.0005626678466797
Epoch [4/10000], Loss: 0.1113328668806288
 Validation Loss: 1.0003389120101929
Epoch [5/10000], Loss: 0.11130756802029079
 Validation Loss: 1.0001153945922852
Epoch [6/10000], Loss: 0.11128232214185926
 Validation Loss: 0.999891996383667
Epoch [7/10000], Loss: 0.11125708950890435
 Validation Loss: 0.9996685981750488
Epoch [8/10000], Loss: 0.11123184363047282
 Validation Loss: 0.9994453191757202
Epoch [9/10000], Loss: 0.1112066109975179
 Validation Loss: 0.9992221593856812
Epoch [10/10000], Loss: 0.11118140485551622
 Validation Loss: 0.9989992380142212
Epoch [11/10000], Loss: 0.11115622520446777
 Validation Loss: 0.9987762570381165
Epoch [12/10000], Loss: 0.1111310323079427
 Validation Loss: 0.9985538125038147
Epoch [13/10000], Loss: 0.111105899016062

In [227]:
model.eval()  # Set the model to evaluation mode
val_loss = 0.0
start_time = time.time()
with torch.no_grad():
    
    inputs = val_loader[:, ..., :-1].to(device)
    outputs = model(inputs).to(device)
    targets = val_loader[:, ..., -1].to(device)  # Forward pass
    outputs = outputs.squeeze(-1)
    loss = criterion(outputs, targets).to(device)  # Compute the loss
    val_loss += loss.item()
end_time = time.time()

elapsed_time = end_time - start_time
print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss/len(val_loader)}")
print(f"validation took {elapsed_time} secondson {device}.")

Epoch [10000/10000], Validation Loss: 0.35234078764915466
validation took 0.01002192497253418 secondson cuda.
